# Read Data Sample

In [1]:
import pandas as pd
import numpy as np
import os
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")
    
    kdd_train_5labels = pd.read_pickle("dataset/kdd_train_5labels.pkl")
    kdd_test_5labels = pd.read_pickle("dataset/kdd_test_5labels.pkl")
    

In [3]:
dataset.kdd_train_2labels.shape

(125973, 124)

In [4]:
dataset.kdd_test_2labels.shape

(22544, 124)

In [5]:
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['is_Attack','is_Normal']
    
    x_input = dataset.kdd_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.kdd_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.kdd_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.kdd_test_2labels.loc[:,output_columns_2labels]

    ss = pp.StandardScaler()

    x_train = ss.fit_transform(x_input)
    x_test = ss.transform(x_test_input)

    y_train = y_output.values
    y_test = y_test.values

    x_train = np.hstack((x_train, y_train))
    x_test = np.hstack((x_test, np.random.normal(size = (x_test.shape[0], y_train.shape[1]))))
    #x_test = np.hstack((x_test, y_test))
    
preprocess.x_train.std()

0.99186991653217393

In [6]:
import tensorflow as tf


In [7]:
class network(object):
    
    input_dim = 124
    classes = 2
    hidden_encoder_dim = 124
    hidden_layers = 1
    latent_dim = 10

    hidden_decoder_dim = 124
    lam = 0.01
    
    def __init__(self, classes, hidden_layers, num_of_features):
        self.classes = classes
        self.hidden_layers = hidden_layers
        self.latent_dim = num_of_features
            
    def build_layers(self):
        tf.reset_default_graph()
        #learning_rate = tf.Variable(initial_value=0.001)

        input_dim = self.input_dim
        classes = self.classes
        hidden_encoder_dim = self.hidden_encoder_dim
        hidden_layers = self.hidden_layers
        latent_dim = self.latent_dim
        hidden_decoder_dim = self.hidden_decoder_dim
        lam = self.lam
        
        with tf.variable_scope("Input"):
            self.x = tf.placeholder("float", shape=[None, input_dim])
            self.y_ = tf.placeholder("float", shape=[None, classes])
            self.keep_prob = tf.placeholder("float")
        
        with tf.variable_scope("Layer_Encoder"):

            hidden_encoder = tf.layers.dense(self.x, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_encoder = tf.layers.dense(hidden_encoder, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
                
        with tf.variable_scope("Layer_Mean"):
            mu_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = None, kernel_regularizer=tf.nn.l2_loss)

        with tf.variable_scope("Layer_Variance"):
            logvar_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = None, kernel_regularizer=tf.nn.l2_loss)

        with tf.variable_scope("Sampling_Distribution"):
            # Sample epsilon
            epsilon = tf.random_normal(tf.shape(logvar_encoder), mean=0, stddev=1, name='epsilon')

            # Sample latent variable
            std_encoder = tf.exp(0.5 * logvar_encoder)
            z = mu_encoder + tf.multiply(std_encoder, epsilon)
            
            #tf.summary.histogram("Sample_Distribution", z)

        with tf.variable_scope("Layer_Decoder"):
            hidden_decoder = tf.layers.dense(z, hidden_decoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_decoder = tf.nn.dropout(hidden_decoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_decoder = tf.layers.dense(hidden_decoder, hidden_decoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_decoder = tf.nn.dropout(hidden_decoder, self.keep_prob)
                
        with tf.variable_scope("Layer_Reconstruction"):
            self.x_hat = tf.layers.dense(hidden_decoder, input_dim, activation = None)
            
            self.y = tf.slice(self.x_hat, [0,input_dim-2], [-1,-1])

        with tf.variable_scope("Loss"):
            
            BCE = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.x_hat, labels=self.x), reduction_indices=1)
            KLD = -0.5 * tf.reduce_mean(1 + logvar_encoder - tf.pow(mu_encoder, 2) - tf.exp(logvar_encoder), reduction_indices=1)
            softmax_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = self.y_, logits = self.y))

            loss = tf.reduce_mean((BCE + KLD + softmax_loss) * lam)

            #loss = tf.clip_by_value(loss, -1e-2, 1e-2)
            #loss = tf.where(tf.is_nan(loss), 1e-2, loss)
            #loss = tf.where(tf.equal(loss, -1e-2), tf.random_normal(loss.shape), loss)
            #loss = tf.where(tf.equal(loss, 1e-2), tf.random_normal(loss.shape), loss)
            
            self.regularized_loss = tf.abs(loss, name = "Regularized_loss")
            correct_prediction = tf.equal(tf.argmax(self.y_, 1), tf.argmax(self.y, 1))
            self.tf_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "Accuracy")

        with tf.variable_scope("Optimizer"):
            learning_rate=1e-2
            optimizer = tf.train.AdamOptimizer(learning_rate)
            gradients, variables = zip(*optimizer.compute_gradients(self.regularized_loss))
            gradients = [
                None if gradient is None else tf.clip_by_value(gradient, -1, 1)
                for gradient in gradients]
            self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            #self.train_op = optimizer.minimize(self.regularized_loss)
            
            
        # add op for merging summary
        #self.summary_op = tf.summary.merge_all()
        self.pred = tf.argmax(self.y, axis = 1)
        self.actual = tf.argmax(self.y_, axis = 1)

        # add Saver ops
        self.saver = tf.train.Saver()
        

In [8]:
import collections

class Train:    
    
    result = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])

    predictions = {}

    results = []
    best_acc = 0
    
    def train(epochs, net, h,f):
        batch_iterations = 200
        os.makedirs("dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_{}_features count_{}".format(epochs,h,f),
                    exist_ok = True)
        with tf.Session() as sess:
            #summary_writer_train = tf.summary.FileWriter('./logs/kdd/VAE/training', graph=sess.graph)
            #summary_writer_valid = tf.summary.FileWriter('./logs/kdd/VAE/validation')

            sess.run(tf.global_variables_initializer())
            
            Train.best_acc = 0
            for epoch in range(1, (epochs+1)):
                #print("Step {} | Training Loss:".format(epoch), end = " " )
                x_train, x_valid, y_train, y_valid, = ms.train_test_split(preprocess.x_train, 
                                                                          preprocess.y_train, 
                                                                          test_size=0.1)
                batch_indices = np.array_split(np.arange(x_train.shape[0]), 
                                           batch_iterations)
                                                                          
                for i in batch_indices:
                    _, train_loss = sess.run([net.train_op, 
                                                           net.regularized_loss, 
                                                           ], #net.summary_op
                                                          feed_dict={net.x: x_train[i,:], 
                                                                     net.y_: y_train[i,:], 
                                                                     net.keep_prob:1})
                    if((train_loss > 1e4 or np.isnan(train_loss)) and epoch > 1):
                        print("Step {} | High Training Loss: {:.6f} ... Restoring Net".format(epoch, train_loss))
                        net.saver.restore(sess, 
                                          tf.train.latest_checkpoint('dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_{}_features count_{}'
                                                                     .format(epochs,h,f)))
                    
                    #summary_writer_train.add_summary(summary_str, epoch)
                    #if(train_loss > 1e9):
                    
                    #print("{:.6f}".format(train_loss), end = ", " )
                    
                #print("")
                valid_loss, valid_accuracy = sess.run([net.regularized_loss, net.tf_accuracy], feed_dict={net.x: x_valid, 
                                                                     net.y_: y_valid, 
                                                                     net.keep_prob:1})
                    
                
                accuracy, test_loss, pred_value, actual_value, y_pred = sess.run([net.tf_accuracy, net.regularized_loss, 
                                                               net.pred, 
                                                               net.actual, net.y], 
                                                              feed_dict={net.x: preprocess.x_test, 
                                                                         net.y_: preprocess.y_test, 
                                                                         net.keep_prob:1})
                #print("*************** \n")
                print("Step {} | Training Loss: {:.6f} | Test Loss: {:6f} | Test Accuracy: {:.6f}".format(epoch, train_loss, test_loss, accuracy))
                #print("*************** \n")
                #print("Accuracy on Test data: {}".format(accuracy))

                

                if accuracy > Train.best_acc:
                    Train.best_acc = accuracy
                    Train.pred_value = pred_value
                    Train.actual_value = actual_value
                    Train.best_parameters = "Hidden Layers:{}, Features Count:{}".format(h, f)
                   
                    #net.saver.save(sess, "dataset/tf_vae_only_vae_loss_nsl_kdd_hidden layers_{}_features count_{}".format(epochs,h,f))
                    #Train.results.append(Train.result(epochs, f, h,valid_accuracy, accuracy))
                    #curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,-2], "Normal_prob":y_pred[:, -1]})
                    #Train.predictions.update({"{}_{}_{}".format(epochs,f,h):curr_pred})
            
                    net.saver.save(sess, 
                                   "dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_{}_features count_{}/model"
                                   .format(epochs,h,f), 
                                   global_step = epoch, 
                                   write_meta_graph=False)
                    
                    curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,-2], "Normal_prob":y_pred[:, -1]})
                    Train.predictions.update({"{}_{}_{}".format(epochs,f,h):
                                              (curr_pred, 
                                               Train.result(epochs, f, h,valid_accuracy, accuracy))})
            
            

In [9]:
import itertools
class Hyperparameters:
#    features_arr = [2, 4, 8, 16, 32, 64, 128, 256]
#    hidden_layers_arr = [2, 4, 6, 10]
    features_arr = [4, 8, 16, 32]
    hidden_layers_arr = [2, 4]

    epochs = [100]
    
    for e, h, f in itertools.product(epochs, hidden_layers_arr, features_arr):
        print("Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
        n = network(2,h,f)
        n.build_layers()
        Train.train(e, n, h,f)
        

Current Layer Attributes - epochs:100 hidden layers:2 features count:4
Step 1 | Training Loss: 0.000812 | Test Loss: 0.021126 | Test Accuracy: 0.770183
Step 2 | Training Loss: 0.000887 | Test Loss: 0.009059 | Test Accuracy: 0.733410
Step 3 | Training Loss: 0.000701 | Test Loss: 0.008002 | Test Accuracy: 0.792938
Step 4 | Training Loss: 0.000475 | Test Loss: 0.006612 | Test Accuracy: 0.783180
Step 5 | Training Loss: 0.000612 | Test Loss: 0.007014 | Test Accuracy: 0.790144
Step 6 | Training Loss: 0.000298 | Test Loss: 0.008389 | Test Accuracy: 0.762021
Step 7 | Training Loss: 0.000662 | Test Loss: 0.007873 | Test Accuracy: 0.769473
Step 8 | Training Loss: 0.001378 | Test Loss: 0.008335 | Test Accuracy: 0.801233
Step 9 | Training Loss: 0.000478 | Test Loss: 0.010312 | Test Accuracy: 0.772445
Step 10 | Training Loss: 0.000484 | Test Loss: 0.008188 | Test Accuracy: 0.742681
Step 11 | Training Loss: 0.000420 | Test Loss: 0.009454 | Test Accuracy: 0.722055
Step 12 | Training Loss: 0.000585 | 

Step 1 | Training Loss: 0.000742 | Test Loss: 0.028600 | Test Accuracy: 0.788325
Step 2 | High Training Loss: 9697523400704.000000 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 2 | Training Loss: 0.000537 | Test Loss: 0.030021 | Test Accuracy: 0.783046
Step 3 | Training Loss: 0.000659 | Test Loss: 0.029454 | Test Accuracy: 0.705554
Step 4 | High Training Loss: 9603684237312.000000 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 4 | High Training Loss: 36039.125000 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 4 | Training Loss: 0.001078 | Test Loss: 0.012868 | Test Accuracy: 0.734741
Step 5 | Training Loss: 0.000870 | Test Loss: 0.012154 | Test Accuracy: 0.748847
Step 6 | Training Loss: 0.001081 

Step 83 | Training Loss: 0.000037 | Test Loss: 0.010079 | Test Accuracy: 0.737402
Step 84 | Training Loss: 0.000909 | Test Loss: 0.010299 | Test Accuracy: 0.741306
Step 85 | Training Loss: 0.000518 | Test Loss: 0.012035 | Test Accuracy: 0.740907
Step 86 | Training Loss: 0.000263 | Test Loss: 0.011164 | Test Accuracy: 0.732967
Step 87 | Training Loss: 0.000301 | Test Loss: 0.012338 | Test Accuracy: 0.715135
Step 88 | Training Loss: 0.000061 | Test Loss: 0.010113 | Test Accuracy: 0.745254
Step 89 | Training Loss: 0.000570 | Test Loss: 0.009382 | Test Accuracy: 0.762509
Step 90 | Training Loss: 0.000269 | Test Loss: 0.010990 | Test Accuracy: 0.752218
Step 91 | Training Loss: 0.000026 | Test Loss: 0.012137 | Test Accuracy: 0.758384
Step 92 | Training Loss: 0.000249 | Test Loss: 0.012146 | Test Accuracy: 0.733943
Step 93 | Training Loss: 0.000013 | Test Loss: 0.013561 | Test Accuracy: 0.720680
Step 94 | Training Loss: 0.000043 | Test Loss: 0.012893 | Test Accuracy: 0.731592
Step 95 | Traini

Step 71 | Training Loss: 0.000177 | Test Loss: 0.010299 | Test Accuracy: 0.762553
Step 72 | Training Loss: 0.000434 | Test Loss: 0.010862 | Test Accuracy: 0.772312
Step 73 | Training Loss: 0.000031 | Test Loss: 0.011233 | Test Accuracy: 0.760735
Step 74 | Training Loss: 0.000005 | Test Loss: 0.010820 | Test Accuracy: 0.771292
Step 75 | Training Loss: 0.000106 | Test Loss: 0.010669 | Test Accuracy: 0.776881
Step 76 | Training Loss: 0.000112 | Test Loss: 0.010365 | Test Accuracy: 0.760956
Step 77 | Training Loss: 0.000592 | Test Loss: 0.013542 | Test Accuracy: 0.790055
Step 78 | Training Loss: 0.000127 | Test Loss: 0.010491 | Test Accuracy: 0.766368
Step 79 | Training Loss: 0.000463 | Test Loss: 0.010972 | Test Accuracy: 0.764638
Step 80 | Training Loss: 0.000906 | Test Loss: 0.014084 | Test Accuracy: 0.799015
Step 81 | Training Loss: 0.000058 | Test Loss: 0.010741 | Test Accuracy: 0.759315
Step 82 | Training Loss: 0.000212 | Test Loss: 0.010921 | Test Accuracy: 0.763662
Step 83 | Traini

Step 2 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 2 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 2 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 2 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 2 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 2 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidde

...

Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd

Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd

Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd

Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd/hidden layers_100_features count_2/model-1
Step 37 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_vae_loss_nsl_kdd

KeyboardInterrupt: 

In [ ]:
dict1 = {}
dict2 = []
for k, (v1, v2) in Train.predictions.items():
    dict1.update({k: v1})
    dict2.append(v2)
    

In [ ]:
Train.predictions = dict1
Train.results = dict2

In [ ]:
df_results = pd.DataFrame(Train.results)

In [ ]:
df_results.sort_values(by = 'test_score', ascending = False)

In [ ]:
pd.Panel(Train.predictions).to_pickle("dataset/tf_vae_only_vae_loss_nsl_kdd_predictions.pkl")
df_results.to_pickle("dataset/tf_vae_only_vae_loss_nsl_kdd_scores.pkl")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    np.set_printoptions(precision=4)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j].round(4),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot(actual_value, pred_value):
    from sklearn.metrics import confusion_matrix
    cm_2labels = confusion_matrix(y_pred = pred_value, y_true = actual_value)
    plt.figure(figsize=[6,6])
    plot_confusion_matrix(cm_2labels, preprocess.output_columns_2labels, normalize = True,
                         title = Train.best_parameters)

In [ ]:
plot(actual_value = Train.actual_value, pred_value = Train.pred_value)